In [1]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
spark = SparkSession.builder \
    .master("local[*]") \
    .appName('test') \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/24 12:18:25 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/24 12:18:26 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [2]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [7]:
rdd = df_green \
    .select('lpep_pickup_datetime', 'PULocationID', 'total_amount') \
    .rdd

In [8]:
rdd.take(5)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 20, 15, 9), PULocationID=67, total_amount=33.45),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 15, 20, 23, 41), PULocationID=260, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 5, 16, 32, 26), PULocationID=82, total_amount=8.3),
 Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 29, 19, 22, 42), PULocationID=166, total_amount=12.74)]

In [ ]:
#Making the select as RDD commands
Select 
    PULocationID as zone,
    date_trunc('hour', cast(lpep_pickup_datetime as TIMESTAMP)) as hour,    
    sum(total_amount) as amount,
    count(1) as number_of_records
from 
    green
where 
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
group by 
    1,2
Order by
    zone, hour


In [9]:
#Where
from datetime import datetime

In [12]:
start = datetime(year=2020, month=1, day=1)

def filter_outliers(row):
    return row.lpep_pickup_datetime >= start

In [14]:
#rdd.filter(lambda row: row.lpep_pickup_datetime >= start)
rdd.filter(filter_outliers).take(1)

[Row(lpep_pickup_datetime=datetime.datetime(2020, 1, 23, 13, 10, 15), PULocationID=74, total_amount=44.97)]

In [19]:
#Group By
#rdd.filter(lambda row: row.lpep_pickup_datetime >= start)
def prepare_group(row):
    hour = row.lpep_pickup_datetime.replace(minute=0, second=0, microsecond=0)
    zone = row.PULocationID
    key = (hour, zone)
    
    amount = row.total_amount
    count = 1
    value = (amount,count)
    
    return (key, value)


In [20]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_group).take(5)

[((datetime.datetime(2020, 1, 23, 13, 0), 74), (44.97, 1)),
 ((datetime.datetime(2020, 1, 20, 15, 0), 67), (33.45, 1)),
 ((datetime.datetime(2020, 1, 15, 20, 0), 260), (8.3, 1)),
 ((datetime.datetime(2020, 1, 5, 16, 0), 82), (8.3, 1)),
 ((datetime.datetime(2020, 1, 29, 19, 0), 166), (12.74, 1))]

In [23]:
def calculate_revenue(left_value, right_value):
    left_amount, left_count = left_value
    right_amount, right_count = right_value

    output_amount = left_amount + right_amount
    output_count = left_count + right_count
    
    return (output_amount, output_count)
                

In [24]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_group) \
    .reduceByKey(calculate_revenue) \
    .take(5)

[((datetime.datetime(2020, 1, 15, 20, 0), 260), (163.90000000000003, 14)),
 ((datetime.datetime(2020, 1, 29, 19, 0), 166), (695.0099999999999, 45)),
 ((datetime.datetime(2020, 1, 16, 8, 0), 41), (736.1399999999996, 54)),
 ((datetime.datetime(2020, 1, 4, 20, 0), 129), (583.27, 38)),
 ((datetime.datetime(2020, 1, 2, 8, 0), 66), (197.69, 10))]

In [32]:
# UNNESTING
from collections import namedtuple
RevenueRow = namedtuple('RevenueRow', ['hour', 'zone', 'revenue', 'count'])
def unnest_data(row):
    return RevenueRow(
        hour=row[0][0], 
        zone=row[0][1], 
        revenue=row[1][0], 
        count=row[1][1])


In [33]:
rdd \
    .filter(filter_outliers) \
    .map(prepare_group) \
    .reduceByKey(calculate_revenue) \
    .map(unnest_data) \
    .take(5)

[RevenueRow(hour=datetime.datetime(2020, 1, 15, 20, 0), zone=260, revenue=163.90000000000003, count=14),
 RevenueRow(hour=datetime.datetime(2020, 1, 29, 19, 0), zone=166, revenue=695.0099999999999, count=45),
 RevenueRow(hour=datetime.datetime(2020, 1, 16, 8, 0), zone=41, revenue=736.1399999999996, count=54),
 RevenueRow(hour=datetime.datetime(2020, 1, 4, 20, 0), zone=129, revenue=583.27, count=38),
 RevenueRow(hour=datetime.datetime(2020, 1, 2, 8, 0), zone=66, revenue=197.69, count=10)]

In [ ]:
#Convert Now To Dataframe

In [35]:
# UNNESTING & Tunning Schema
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_group) \
    .reduceByKey(calculate_revenue) \
    .map(unnest_data) \
    .toDF()

In [36]:
df_result.schema

StructType([StructField('hour', TimestampType(), True), StructField('zone', LongType(), True), StructField('revenue', DoubleType(), True), StructField('count', LongType(), True)])

In [37]:
from pyspark.sql import types

result_schema = types.StructType([
    types.StructField('hour', types.TimestampType(), True), 
    types.StructField('zone', types.IntegerType(), True), 
    types.StructField('revenue', types.DoubleType(), True), 
    types.StructField('count', types.IntegerType(), True)])

In [38]:
# UNNESTING & Tunning Schema
df_result = rdd \
    .filter(filter_outliers) \
    .map(prepare_group) \
    .reduceByKey(calculate_revenue) \
    .map(unnest_data) \
    .toDF(result_schema)

In [39]:
df_result.write.parquet('tmp/rdd_green_revenue')

In [ ]:
#MapPartition

In [47]:
columns = ['VendorID', 'lpep_pickup_datetime', 'PULocationID', 'DOLocationID', 'trip_distance']
duration_rdd = df_green \
    .select(columns) \
    .rdd

In [59]:
import pandas as pd

In [58]:
#model = ...
def model_predict(df):
    #y_pred = model.predict(df)
    y_pred = df.trip_distance * 5
    return y_pred

In [57]:
def apply_model_in_batch(rows):
    df = pd.DataFrame(rows, columns=columns)
    predictions = model_predict(df)
    df['predicted_duration'] =  predictions
    for row in df.itertuples():
        yield row

In [65]:
df_predicts = duration_rdd.mapPartitions(apply_model_in_batch).toDF().drop('Index')

In [67]:
df_predicts.select('predicted_duration').show()

+------------------+
|predicted_duration|
+------------------+
|63.849999999999994|
|              40.0|
|              6.35|
|              6.25|
| 9.200000000000001|
|               3.8|
|16.599999999999998|
|             11.05|
|               4.5|
|              30.5|
|               8.7|
|5.8999999999999995|
|              11.0|
|              15.2|
|              4.25|
|25.299999999999997|
|7.8500000000000005|
|              34.0|
| 5.300000000000001|
|              6.15|
+------------------+
only showing top 20 rows

